In [2]:
import pandas as pd

df = pd.read_csv(r"C:\Users\fabri\Desktop\MarcoPatierno_DepositoCorsoPython\env\Giorno 20 30-04\esercitazione\AB_NYC_2019.csv")
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [9]:
df_no_cat = df.drop(columns=['name', 'host_name', 'last_review', 'neighbourhood_group', 'neighbourhood', 'room_type'])

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = df_no_cat.drop(columns=['price'])
y = df_no_cat['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [6]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

def elimina_variabili_vif_pvalue(X_train, y_train,
                                 vif_threshold=10.0,
                                 pvalue_threshold=0.05):
    # Se non è già un DataFrame, lo trasformo
    if not isinstance(X_train, pd.DataFrame):
        X_current = pd.DataFrame(
            X_train,
            columns=[f"x{i}" for i in range(X_train.shape[1])]
        )
    else:
        X_current = X_train.copy()

    while True:
        # Aggiungo costante (ora X_current è DataFrame)
        X_const = sm.add_constant(X_current)
        model   = sm.OLS(y_train, X_const).fit()

        # Prendo i p-value come Series e tolgo la costante
        pvalues = pd.Series(model.pvalues, index=X_const.columns)
        pvalues = pvalues.drop('const')

        # Calcolo il VIF
        vif = pd.DataFrame({
            "Feature": X_current.columns,
            "VIF": [
                variance_inflation_factor(X_current.values, i)
                for i in range(X_current.shape[1])
            ]
        })

        # Unisco VIF e p-value
        stats = vif.copy()
        stats["p-value"] = pvalues.values

        # Seleziono candidati con VIF e p-value alti
        candidates = stats[
            (stats["VIF"] > vif_threshold) &
            (stats["p-value"] > pvalue_threshold)
        ]
        if candidates.empty:
            print("\nNessuna variabile da eliminare. Selezione completata.")
            break

        # Rimuovo la feature peggiore
        worst = candidates.sort_values(by="VIF", ascending=False).iloc[0]
        feat = worst["Feature"]
        print(f"Rimuovo '{feat}' (VIF={worst['VIF']:.2f}, p-value={worst['p-value']:.4f})")

        X_current = X_current.drop(columns=[feat])

    print("\nFeature finali selezionate:")
    print(X_current.columns.tolist())
    return X_current
X_current = elimina_variabili_vif_pvalue(X_train_scaled, y_train, 10.0, 0.05)

NameError: name 'X_train_scaled' is not defined